# Importing Libraries

In [1]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Load environment

In [2]:
environment_name = "CartPole-v1"
env = gym.make(environment_name,render_mode='human')

In [3]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done,_, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
# env.close()

C:\Users\vjana\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:1 Score:43.0
Episode:2 Score:21.0
Episode:3 Score:17.0
Episode:4 Score:15.0
Episode:5 Score:14.0


In [4]:
env.close()

# Training and Log visualization

In [5]:
log_path=os.path.join("Trainings","Logs")

In [6]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1,tensorboard_log=log_path)

C:\Users\vjana\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cuda device


In [7]:
model.learn(total_timesteps=20000)

Logging to Trainings\Logs\PPO_1


C:\Users\vjana\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


-----------------------------
| time/              |      |
|    fps             | 274  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 200         |
|    iterations           | 2           |
|    time_elapsed         | 20          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008582674 |
|    clip_fraction        | 0.116       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00201     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.66        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0191     |
|    value_loss           | 50.6        |
-----------------------------------------
----------------------------------

In [8]:
PPO_path = os.path.join('Trainings', 'Saved Models', 'PPO_model')
model.save(PPO_path)

C:\Users\vjana\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:284: UserWarning: Path 'Trainings\Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [9]:
del model

In [10]:
model=PPO.load(PPO_path,env=env)

In [11]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\vjana\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\vjana\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(500.0, 0.0)

In [12]:
env.close()

In [13]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([ 0.42420346,  0.05455754, -0.02704742, -0.25027263], dtype=float32)}]


In [14]:
env.close()

In [15]:
training_log_path = os.path.join(log_path, 'PPO_1')

In [16]:
!tensorboard --logdir={training_log_path}

^C


# Training with Callbacks

In [17]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [25]:
save_path = os.path.join('Trainings', 'Saved Models')
log_path = os.path.join('Trainings', 'Logs')

In [26]:
env = gym.make(environment_name,render_mode='rgb_array')
env = DummyVecEnv([lambda: env])

In [27]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [28]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cuda device


In [29]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Trainings\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 304  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 240         |
|    iterations           | 2           |
|    time_elapsed         | 17          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008516259 |
|    clip_fraction        | 0.0905      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00157    |
|    learning_rate        | 0.0003      |
|    loss                 | 8.26        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0136     |
|    value_loss           | 52.4        |
-----------------------------------------
--

In [30]:
model_path = os.path.join('Trainings', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [31]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(347.8, 125.39122776334874)

In [32]:
env.close()

In [33]:
#Changing policies
# net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
# model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})
# model.learn(total_timesteps=20000, callback=eval_callback)